In [5]:
import json

with open("/Users/shreyasguha/Downloads/Shreyas/code/FinanceGroup2/IndiFinBERT_TextMapping.json", 'r') as f:
  data = json.load(f)

text = ""
for key, value in data.items():
    text += value

print(len(text))

tokens = list(text)
stoi = {ch: idx for idx, ch in enumerate(sorted(set(tokens)))}
ids = [stoi[ch] for ch in tokens]
itos = {v: k for k, v in stoi.items()}

print(len(tokens))
print(len(stoi))

10204311
10204311
205


In [8]:
import regex as re
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

regex_text = re.findall(gpt2pat, text)
print(regex_text[:20])
print(tokens[:20])

['The', ' Indian', ' economy', ' witnessed', ' robust', ' growth', ' during', ' 2006', '-', '07', ' for', ' the', ' fourth', ' year', ' in', ' succession', '.', ' According', ' to', ' the']
['T', 'h', 'e', ' ', 'I', 'n', 'd', 'i', 'a', 'n', ' ', 'e', 'c', 'o', 'n', 'o', 'm', 'y', ' ', 'w']


In [9]:
regex_id = []
for i in range(len(regex_text)):
    l = []
    word = regex_text[i]
    for j in range(len(word)):
        l.append(stoi[word[j]])
    regex_id.append(l)

print(regex_id[:20])

[[53, 72, 69], [1, 42, 78, 68, 73, 65, 78], [1, 69, 67, 79, 78, 79, 77, 89], [1, 87, 73, 84, 78, 69, 83, 83, 69, 68], [1, 82, 79, 66, 85, 83, 84], [1, 71, 82, 79, 87, 84, 72], [1, 68, 85, 82, 73, 78, 71], [1, 19, 17, 17, 23], [14], [17, 24], [1, 70, 79, 82], [1, 84, 72, 69], [1, 70, 79, 85, 82, 84, 72], [1, 89, 69, 65, 82], [1, 73, 78], [1, 83, 85, 67, 67, 69, 83, 83, 73, 79, 78], [15], [1, 34, 67, 67, 79, 82, 68, 73, 78, 71], [1, 84, 79], [1, 84, 72, 69]]


In [14]:
def get_stats(ids):
    counts = {}
    for i in range(len(ids)):
      for pair in zip(ids[i], ids[i][1:]):
          counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids_l, pair, idx):
  newids = []
  for j in range(len(ids_l)):
    ids = ids_l[j]
    n = []
    i = 0
    while i < len(ids):
      if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
        n.append(idx)
        i += 2
      else:
        n.append(ids[i])
        i += 1
    newids.append(n)
  return newids

# ---
vocab_size = 3000 # the desired final vocabulary size
num_merges = vocab_size - 205
ids = list(regex_id) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 205 + i
  # print(f"merging {pair} with {stats[pair]} occurances into a new token {idx}")
  print(i, stats[pair])
  ids = merge(ids, pair, idx)
  merges[pair] = idx
#   vocab_tokens.append(idx)
#   vocab.append()

0 208863
1 175783
2 141159
3 135245
4 134724
5 118677
6 110821
7 100616
8 93488
9 89667
10 88095
11 86944
12 86377
13 80924
14 77824
15 72436
16 71651
17 67395
18 64769
19 60039
20 59586
21 59390
22 58690
23 57535
24 56879
25 55609
26 55574
27 53170
28 52864
29 50128
30 48210
31 48180
32 47734
33 47160
34 47036
35 44330
36 43380
37 42065
38 41837
39 41834
40 40571
41 35482
42 32463
43 32349
44 32260
45 31793
46 28336
47 28031
48 27699
49 27046
50 27034
51 26133
52 25873
53 24886
54 24822
55 24014
56 22913
57 22850
58 22039
59 21742
60 20864
61 20759
62 20417
63 19272
64 19177
65 19144
66 18623
67 18605
68 18514
69 18341
70 17583
71 17255
72 17201
73 16748
74 16280
75 16261
76 16223
77 16141
78 16051
79 15990
80 15750
81 15624
82 15477
83 15247
84 15121
85 14941
86 14555
87 14345
88 14339
89 14229
90 14094
91 13988
92 13569
93 13511
94 13047
95 13045
96 12887
97 12716
98 12301
99 12203
100 12177
101 12093
102 12063
103 11887
104 11870
105 11852
106 11708
107 11601
108 11530
109 11395
11

In [15]:
print(len(itos))
print(len(merges))

205
2795


In [16]:
# vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    itos[idx] = itos[p0] + itos[p1]

In [18]:
s = "shreyas"
print(s[1:])

hreyas


In [21]:
vocab = []
for key, value in itos.items():
    ex = value
    if value[0] == " ":
        vocab.append(value[1:])
    else:
        vocab.append(value)

print(vocab)

['\t', '', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '£', '¥', '§', '°', '±', 'µ', '¼', '½', 'Ç', 'Ö', '×', 'à', 'á', 'ä', 'ç', 'è', 'é', 'í', 'ñ', '÷', 'ı', 'š', 'ˆ', '̂', 'Δ', 'Σ', 'Ψ', 'α', 'β', 'γ', 'δ', 'ε', 'θ', 'λ', 'π', 'ρ', 'σ', 'φ', 'ं', 'ए', 'क', 'ग', 'च', 'ट', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'स', 'ा', 'ि', 'ी', 'ु', 'े', 'ै', 'ो', '्', '\u2002', '–', '—', '‘', '’', '“', '”', '„', '‟', '†', '‡', '•', '…', '€', '₹', '∆', '∈', '∑', '−', '∗', '√', '≠', '≤', '⊗', '●', '\uf020', '\uf044', '\uf050', '\uf057', '\uf062', '\uf065', '\uf06c', '\uf

In [80]:
merges_cp = merges

In [81]:
for i in range(100):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 3000 + i
  # print(f"merging {pair} with {stats[pair]} occurances into a new token {idx}")
  print(i, stats[pair])
  ids = merge(ids, pair, idx)
  merges[pair] = idx

0 257
1 257
2 257
3 257
4 256
5 256
6 256
7 256
8 256
9 256
10 256
11 256
12 256
13 256
14 255
15 255
16 255
17 255
18 255
19 254
20 254
21 254
22 254
23 254
24 254
25 254
26 254
27 253
28 253
29 253
30 253
31 252
32 252
33 252
34 252
35 252
36 252
37 252
38 252
39 252
40 252
41 251
42 251
43 251
44 251
45 251
46 250
47 250
48 250
49 250
50 250
51 250
52 250
53 250
54 250
55 250
56 250
57 250
58 250
59 250
60 249
61 249
62 249
63 249
64 249
65 249
66 249
67 248
68 248
69 248
70 248
71 248
72 248
73 247
74 247
75 247
76 247
77 247
78 247
79 247
80 247
81 247
82 246
83 246
84 246
85 246
86 246
87 246
88 246
89 246
90 246
91 246
92 246
93 246
94 246
95 246
96 245
97 245
98 245
99 245


In [82]:
# vocab = {idx: bytes([idx]) for idx in range(256)}
itos = {v: k for k, v in reverse_vocab.items()}
for (p0, p1), idx in merges.items():
    itos[idx] = itos[p0] + itos[p1]

In [83]:
for key, value in itos.items():
    print(key, value)

0 	
1  
2 !
3 "
4 #
5 $
6 %
7 &
8 '
9 (
10 )
11 *
12 +
13 ,
14 -
15 .
16 /
17 0
18 1
19 2
20 3
21 4
22 5
23 6
24 7
25 8
26 9
27 :
28 ;
29 <
30 =
31 >
32 ?
33 @
34 A
35 B
36 C
37 D
38 E
39 F
40 G
41 H
42 I
43 J
44 K
45 L
46 M
47 N
48 O
49 P
50 Q
51 R
52 S
53 T
54 U
55 V
56 W
57 X
58 Y
59 Z
60 [
61 ]
62 ^
63 _
64 `
65 a
66 b
67 c
68 d
69 e
70 f
71 g
72 h
73 i
74 j
75 k
76 l
77 m
78 n
79 o
80 p
81 q
82 r
83 s
84 t
85 u
86 v
87 w
88 x
89 y
90 z
91 {
92 |
93 }
94 ~
95 £
96 ¥
97 §
98 °
99 ±
100 µ
101 ¼
102 ½
103 Ç
104 Ö
105 ×
106 à
107 á
108 ä
109 ç
110 è
111 é
112 í
113 ñ
114 ÷
115 ı
116 š
117 ˆ
118 ̂
119 Δ
120 Σ
121 Ψ
122 α
123 β
124 γ
125 δ
126 ε
127 θ
128 λ
129 π
130 ρ
131 σ
132 φ
133 ं
134 ए
135 क
136 ग
137 च
138 ट
139 न
140 प
141 फ
142 ब
143 भ
144 म
145 य
146 र
147 ल
148 व
149 श
150 स
151 ा
152 ि
153 ी
154 ु
155 े
156 ै
157 ो
158 ्
159  
160 –
161 —
162 ‘
163 ’
164 “
165 ”
166 „
167 ‟
168 †
169 ‡
170 •
171 …
172 €
173 ₹
174 ∆
175 ∈
176 ∑
177 −
178 ∗
179 √
180 ≠
181 ≤
182 ⊗
183 ●
184 


In [84]:
vocab = []
for key, value in itos.items():
    vocab.append(value)

print(len(vocab))
print(vocab)

3100
['\t', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '£', '¥', '§', '°', '±', 'µ', '¼', '½', 'Ç', 'Ö', '×', 'à', 'á', 'ä', 'ç', 'è', 'é', 'í', 'ñ', '÷', 'ı', 'š', 'ˆ', '̂', 'Δ', 'Σ', 'Ψ', 'α', 'β', 'γ', 'δ', 'ε', 'θ', 'λ', 'π', 'ρ', 'σ', 'φ', 'ं', 'ए', 'क', 'ग', 'च', 'ट', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'स', 'ा', 'ि', 'ी', 'ु', 'े', 'ै', 'ो', '्', '\u2002', '–', '—', '‘', '’', '“', '”', '„', '‟', '†', '‡', '•', '…', '€', '₹', '∆', '∈', '∑', '−', '∗', '√', '≠', '≤', '⊗', '●', '\uf020', '\uf044', '\uf050', '\uf057', '\uf062', '\uf065', '\uf06c'

In [22]:
# List of strings
# lines = ["This is line 1", "This is line 2", "This is line 3"]

# Write to a .txt file
with open("/Users/shreyasguha/Downloads/Shreyas/code/FinanceGroup2/indic_vocab_3000_regex.txt", "w", encoding="utf-8") as f:
    for line in vocab:
        f.write(line + "\n")  # Add newline after each line